###### ===================================================================================================================

# HTTP Requesting & Dumping File

###### ===================================================================================================================

### Run Commands

In [1]:
%run "C:\\Users\DELL\\Desktop\\Projects\\Wealth Management System\\Python\\General Files\\Libraries.ipynb"

In [2]:
%run "C:\\Users\DELL\\Desktop\\Projects\\Wealth Management System\\Python\\General Files\\SQL_Connection_File.ipynb"

### Class: Request Bond Data

In [3]:
class Request_Bond_Data:
    def __init__(self, constructor_link, date):
        """
        Initializes the Bond_Data class.

        Args:
            constructor_link (str): The base URL for constructing the download link.
            date (str): The date for which data should be fetched.
        """
        self.constructor_link = constructor_link
        self.date = date
        
        self.url = constructor_link + date + '.zip'
        self.csv_file_name_1 = 'icdm' + date + '.csv'
        self.csv_file_name_2 = 'wdm' + date + '.csv'

    def Fetching(self):
        """
        Fetches and processes bond data.

        Returns:
            pd.DataFrame: A DataFrame containing the fetched data.
        """
        try:
            encoded_url = urllib.parse.quote(self.url, safe=':/')

            # Send a GET request to download the zip file
            response = requests.get(encoded_url, headers={'User-Agent': 'Mozilla/5.0'})
                
            if response.status_code == 200:
                # Save the zip file locally
                with open('downloaded_bond_data.zip', 'wb') as file:
                    file.write(response.content)

                # Extract the contents of the zip file to a folder
                with zipfile.ZipFile('downloaded_bond_data.zip', 'r') as zip_ref:
                    zip_ref.extractall('extracted_bond_data')

                # Specify the directory where the CSV files are located
                folder_path = 'extracted_bond_data'
                
                # Check if both CSV files exist in the folder
                if (self.csv_file_name_1 in os.listdir(folder_path)) and (self.csv_file_name_2 in os.listdir(folder_path)):
                    # Create the full path to the CSV files
                    csv_file_path_1 = os.path.join(folder_path, self.csv_file_name_1)
                    csv_file_path_2 = os.path.join(folder_path, self.csv_file_name_2)

                    # Read both CSV files into pandas DataFrames
                    dataframe_1 = pd.read_csv(csv_file_path_1, on_bad_lines='skip')
                    dataframe_2 = pd.read_csv(csv_file_path_2, on_bad_lines='skip')
                    
                    # Check if the DataFrames have more columns than expected (e.g., 12 columns)
                    expected_columns_dataframe_1 = [
                        'Security Code', 'Issuer Name', 
                        'Coupon (%)', 'LTP', 
                        'Weighted Average Price', 
                        'Weighted Average Yield', 
                        'Maturity Date', 
                        'Turnover (Rs. Lakh)', 
                        'ISIN No.', 'Face Value'
                    ]
                    
                    expected_columns_dataframe_2 = [
                        'Scrip Code',
                        'Security Code',
                        'Security Description',
                        'Coupon Rate (%)',
                        'Open Price',
                        'High Price',
                        'Low Price',
                        'Close Price',
                        'No. of Trades',
                        'Total Trade Volume (Face Value in Rs. Lakh)',
                        'Total Trade Turnover (Rs. Lakh)',
                        'MaturityDate'
                    ]
                    
                    # Check dataframe_1 for extra columns
                    extra_columns_1 = [col for col in dataframe_1.columns if col not in expected_columns_dataframe_1]
                    if extra_columns_1:
                        # Removing extra columns from the DataFrame
                        dataframe_1 = dataframe_1[expected_columns_dataframe_1]
                    
                    # Check dataframe_2 for extra columns
                    extra_columns_2 = [col for col in dataframe_2.columns if col not in expected_columns_dataframe_2]
                    if extra_columns_2:
                        # Removing extra columns from the DataFrame
                        dataframe_2 = dataframe_2[expected_columns_dataframe_2]
                    
                    # Dropping the redundant columns
                    dataframe_1.drop(
                        ['Weighted Average Price', 
                         'Weighted Average Yield', 
                         'ISIN No.', 
                         'Face Value'], 
                        axis=1, inplace=True)
                    
                    dataframe_2.drop(
                        ['Scrip Code', 
                         'Open Price', 
                         'High Price', 
                         'Low Price', 
                         'No. of Trades', 
                         'Total Trade Volume (Face Value in Rs. Lakh)'], 
                        axis=1, inplace=True)
                    
                    # Adding columns to both the DataFrames
                    date_to_fill = datetime.strptime(self.date, "%d%m%Y")
                    dataframe_1['Date'] = date_to_fill
                    dataframe_2['Date'] = date_to_fill
                    
                    # Renaming the columns
                    dataframe_1.rename(
                        columns={'Security Code': 'SecurityCode',
                                 'Issuer Name': 'SecurityName',
                                 'Coupon (%)': 'CouponRate',
                                 'LTP': 'LastTradedPrice',
                                 'Maturity Date': 'MaturityDate',
                                 'Turnover (Rs. Lakh)': 'NetTurnover'},
                        inplace=True)

                    dataframe_2.rename(
                        columns={'Security Code': 'SecurityCode',
                                 'Security Description': 'SecurityName',
                                 'Coupon Rate (%)': 'CouponRate',
                                 'Close Price': 'LastTradedPrice',
                                 'Total Trade Turnover (Rs. Lakh)': 'NetTurnover'},
                        inplace=True)

                    # Sort both the DataFrames to have columns in an alphabetically ascending order
                    columns_in_ascending_order = sorted(dataframe_1.columns)
                    dataframe_1 = dataframe_1[columns_in_ascending_order]
                    
                    columns_in_ascending_order = sorted(dataframe_2.columns)
                    dataframe_2 = dataframe_2[columns_in_ascending_order]
                    
                    # Concatenate the two DataFrames
                    concatenated_dataframe = pd.concat([dataframe_1, dataframe_2], axis=0, ignore_index=True)
                    concatenated_dataframe['MaturityDate'] = pd.to_datetime(concatenated_dataframe['MaturityDate'], errors='coerce')
                    
                    return concatenated_dataframe
                else:
                    return pd.DataFrame()
            else:
                return pd.DataFrame()

        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred while fetching the Bonds Data: {e}")

    @staticmethod
    def Bond_Price_Data(constructor_link, sql_connector, table_name, working_dates):
        """
        Executes the bond data fetching and dumping process.

        Args:
            constructor_link (str): The base URL for constructing the download link.
            sql_connector (SQL): An instance of the SQL class for database connection.
            table_name (str): The name of the SQL table to dump data into.
            working_dates (list): A list of dates for which data should be fetched and dumped.
        """
        try:
            for date in working_dates:
                # Initialize the Request_Bond_Data class instance
                request = Request_Bond_Data(constructor_link, date)
                dataframe = request.Fetching()
                
                # Performing the dumping only for dataframes with data
                if not dataframe.empty:
                    sql_connector.Append_SQL_Table(table_name=table_name, current_date=None, dataframe=dataframe)
                else:
                    pass
                
            print("Executed Successfully!")

        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred while dumping the Bond Price Data: {e}")

### Class: Request Equity Data

In [4]:
class Request_Equity_Data:
    def __init__(self, constructor_link, date):
        """
        Initializes the Equity_Data class.

        Args:
            constructor_link (str): The base URL for constructing the download link.
            date (str): The date for which data should be fetched.
            sql_connector (SQL): An instance of the SQL class for database connection.
            table_name (str): The name of the SQL table to dump data into.
        """
        self.constructor_link = constructor_link
        self.date = date
        
        self.url = constructor_link + date + '.zip'
        self.csv_file_name = 'EQ_ISINCODE_' + date + '.CSV'
        self.current_date = None
        
    def Fetching(self):
        """
        Fetches and processes equity data.

        Returns:
            pd.DataFrame: A DataFrame containing the fetched data.
        """
        try:
            encoded_url = urllib.parse.quote(self.url, safe=':/')

            # Send a GET request to download the zip file
            response = requests.get(encoded_url, headers={'User-Agent': 'Mozilla/5.0'})

            if response.status_code == 200:
                # Save the zip file locally
                with open('downloaded_stock_price_data.zip', 'wb') as file:
                    file.write(response.content)

                # Extract the contents of the zip file to a folder
                with zipfile.ZipFile('downloaded_stock_price_data.zip', 'r') as zip_ref:
                    zip_ref.extractall('extracted_stock_price_data')

                # Specify the directory where the CSV file is located
                folder_path = 'extracted_stock_price_data'

                # Check if the CSV file exists in the folder
                if self.csv_file_name in os.listdir(folder_path):
                    # Create the full path to the CSV file
                    csv_file_path = os.path.join(folder_path, self.csv_file_name)

                    # Read the CSV file into a pandas DataFrame
                    dataframe = pd.read_csv(csv_file_path, on_bad_lines='skip')
                    
                    expected_columns = [
                        'SC_CODE',
                        'SC_NAME',
                        'SC_GROUP',
                        'SC_TYPE',
                        'OPEN',
                        'HIGH',
                        'LOW',
                        'CLOSE',
                        'LAST',
                        'PREVCLOSE',
                        'NO_TRADES',
                        'NO_OF_SHRS',
                        'NET_TURNOV',
                        'TDCLOINDI',
                        'ISIN_CODE',
                        'TRADING_DATE',
                        'FILLER2',
                        'FILLER3'
                    ]
                    
                    # Check DataFrame for extra columns
                    extra_columns = [col for col in dataframe.columns if col not in expected_columns]
                    if extra_columns:
                        # Removing extra columns from the DataFrame
                        dataframe = dataframe[expected_columns]
                    
                    # Renaming the columns
                    dataframe.rename(
                        columns={'SC_CODE': 'SCCode',
                                 'SC_NAME': 'Ticker',
                                 'OPEN': 'OpenPrice',
                                 'HIGH': 'HighPrice',
                                 'LOW': 'LowPrice',
                                 'CLOSE': 'ClosePrice',
                                 'LAST': 'LastPrice',
                                 'PREVCLOSE': 'PreviousClose',
                                 'NO_TRADES': 'NumberOfTrades',
                                 'NO_OF_SHRS': 'NumberOfShares',
                                 'NET_TURNOV': 'NetTurnover',
                                 'ISIN_CODE': 'ISINCode',
                                 'TRADING_DATE': 'Date'},
                        inplace=True)
                    
                    # Define a list of columns that need data type conversion
                    columns_to_convert_to_numeric = [
                        'SCCode', 
                        'OpenPrice', 
                        'HighPrice', 
                        'LowPrice', 
                        'ClosePrice', 
                        'LastPrice', 
                        'PreviousClose', 
                        'NumberOfTrades', 
                        'NumberOfShares', 
                        'NetTurnover'
                    ]
                    
                    dataframe[columns_to_convert_to_numeric] = dataframe[columns_to_convert_to_numeric].apply(pd.to_numeric, errors='coerce')
                    
                    # Dropping the redundant columns
                    dataframe.drop(['SC_GROUP', 'SC_TYPE', 'TDCLOINDI', 'FILLER2', 'FILLER3'], axis=1, inplace=True)
                    
                    return dataframe
                else:
                    return pd.DataFrame()
            else:
                return pd.DataFrame()

        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred while fetching the Stock Price data: {e}")
    
    @staticmethod
    def Stock_Price_Data(constructor_link, sql_connector, table_name, working_dates):
        """
        Executes the equity data fetching and dumping process.

        Args:
            working_dates (list): A list of dates for which data should be fetched and dumped.
        """
        try:
            for date in working_dates:
                # Initialize the Request_Equity_Data class instance
                request = Request_Equity_Data(constructor_link, date)
                dataframe = request.Fetching()
                
                # Performing the dumping only for dataframes with data  
                if not dataframe.empty:
                    sql_connector.Append_SQL_Table(table_name=table_name, current_date=None, dataframe=dataframe)
                else:
                    pass
                
            print("Executed Successfully!")

        except Exception as e:
            # Handle exceptions here, or at least log them for debugging
            print(f"An error occurred during the dumping of the Stock Price Data: {e}")

### Execution

In [5]:
# Function to adjust a date to the nearest weekday if it's a weekend
def Adjust_To_Nearest_Weekday(date):
    """
    Adjusts a given date to the nearest weekday if it falls on a weekend.

    Args:
        date (datetime.date): The date to be adjusted.

    Returns:
        datetime.date: The adjusted date.
    """
    if date.weekday() == 5:  # Saturday
        return date - timedelta(days=1)
    elif date.weekday() == 6:  # Sunday
        return date + timedelta(days=1)
    else:
        return date

# Define a function to check if a date is a working day and not a holiday
def Is_Working_Day(date):
    """
    Checks if a date is a working day and not a holiday.

    Args:
        date (datetime.date): The date to be checked.

    Returns:
        bool: True if the date is a working day and not a holiday, False otherwise.
    """
    return date.weekday() < 5 and date.strftime('%d%m%Y') not in Holidays

In [6]:
# Define the start date (5 years back from today)
start_date = date.today() - timedelta(days=5 * 365)

# Define the end date (today's date)
end_date = date.today()

# Adjust start_date and end_date if they fall on weekends
start_date = Adjust_To_Nearest_Weekday(start_date)
end_date = Adjust_To_Nearest_Weekday(end_date)

In [7]:
# Defining the constructor link
Bond_Constructor_Link = 'https://www.bseindia.com/download/Bhavcopy/Debt/DEBTBHAVCOPY'

# List to store the working dates
Bond_Working_Dates = []

# List of holidays (in 'ddmmyy' format) for avoiding unnecessary requests
Holidays = [
    "260118", "130218", "020318", "290318", "300318", "010518", "150818", "220818",
    "130918", "200918", "021018", "181018", "071118", "081118", "231118", "251218",
    "040319", "210319", "170419", "190419", "290419", "010519", "050619", "120819",
    "150819", "020919", "100919", "021019", "081019", "211019", "281019", "121119",
    "251219", "210220", "100320", "020420", "060420", "140420", "100420", "010520",
    "250520", "021020", "161120", "301120", "251220", "260121", "110321", "290321", 
    "140421", "210421", "130521", "210721", "190821", "100921", "151021", "041121", 
    "051121", "191121", "260123", "070323", "300323", "040423", "070423", "140423", 
    "010523", "290623", "150823", "190923", "021023", "141123", "271123", "251223"
]

# Generate working dates and add them to the list
current_date = start_date
while current_date < end_date:
    if Is_Working_Day(current_date):
        formatted_date = current_date.strftime('%d%m%Y')
        Bond_Working_Dates.append(formatted_date)
    current_date += timedelta(days=1)

In [8]:
# Defining the constructor link
Equity_Constructor_Link = 'https://www.bseindia.com/download/BhavCopy/Equity/EQ_ISINCODE_'

# List to store the working dates
Equity_Working_Dates = []

# List of holidays (in 'ddmmyy' format) for avoiding unnecessary requests
Holidays = [
    "260118", "130218", "020318", "290318", "300318", "010518", "150818", "220818",
    "130918", "200918", "021018", "181018", "071118", "081118", "231118", "251218",
    "040319", "210319", "170419", "190419", "290419", "010519", "050619", "120819",
    "150819", "020919", "100919", "021019", "081019", "211019", "281019", "121119",
    "251219", "210220", "100320", "020420", "060420", "140420", "100420", "010520",
    "250520", "021020", "161120", "301120", "251220", "260121", "110321", "290321", 
    "140421", "210421", "130521", "210721", "190821", "100921", "151021", "041121", 
    "051121", "191121", "260123", "070323", "300323", "040423", "070423", "140423", 
    "010523", "290623", "150823", "190923", "021023", "141123", "271123", "251223"
]

# Generate working dates and add them to the list
current_date = start_date
while current_date <= end_date:
    if Is_Working_Day(current_date):
        formatted_date = current_date.strftime('%d%m%y')
        Equity_Working_Dates.append(formatted_date)
    current_date += timedelta(days=1)

In [9]:
# Defining the table name
Stock_Price_Data_Table_Name = "StockPriceData"
Bond_Price_Data_Table_Name = "BondPriceData"

t_start = time.time()

# Execute the data dumping
Request_Bond_Data.Bond_Price_Data(Bond_Constructor_Link, SQL_Connector, Bond_Price_Data_Table_Name, Bond_Working_Dates)
Request_Equity_Data.Stock_Price_Data(Equity_Constructor_Link, SQL_Connector, Stock_Price_Data_Table_Name, Equity_Working_Dates)

t_end = time.time()

print("\nTime taken to dump the HTTP Requested Files: ", round((t_end - t_start) / 3600, 0), " Hours")

Executed Successfully!
Executed Successfully!

Time taken to dump the HTTP Requested Files:  1.0  Hours
